In [53]:
import backtrader as bt
import datetime
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

class ATR_TS_Long(bt.Indicator):
    
    alias = ('ATR_TRAILINGSTOP',)
    
    lines = ('atr_ts',)
    
    params = (
        ('atr_period', 10),
        ('atr_mult', 10),
    )
    
    plotinfo = dict(subplot=False, 
                   )  
    plotlines = dict(
        atr_ts = dict(ls = '-'), 
    )
       
    def __init__(self):
        
        self.atr = btind.ATR(self.data, period = self.params.atr_period)
        self.bigatr = self.atr * self.p.atr_mult 
        
        self.buysell = 1
        self.l.atr_ts = self.data.close - self.bigatr #first calculated value is below close price therefore simulating a long position.
        
        btind.CrossUp(self.data.close, self.l.atr_ts)
        btind.CrossDown(self.data.close, self.l.atr_ts)
        

        
    def next(self):
        
        if self.buysell > 0: 
            self.lines.atr_ts[0] = self.data.close[0] - self.bigatr[0]
            self.l.atr_ts[0] = max(self.l.atr_ts[-1], self.l.atr_ts[0])
            if self.lines.atr_ts > self.data.close: 
                self.lines.atr_ts[0] = self.data.close[0] + self.bigatr[0]
                self.buysell = -1

        elif self.buysell < 0:
            self.lines.atr_ts[0] = self.data.close[0] + self.bigatr[0]
            self.l.atr_ts[0] = min(self.l.atr_ts[-1], self.l.atr_ts[0])
            if self.lines.atr_ts < self.data.close:
                self.l.atr_ts[0] = self.data.close[0] - self.bigatr[0]
                self.buysell = 1

In [54]:
class ATRTS1D(bt.Strategy):
    
    params = (
        ('targ', 470),
        ('stop', 500),
        ('printlog', False),
    )
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
        #Active Buy/Sell order submitted/accepted - Nothing to do
            return
    #Check if an order has been completed
    #Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():             
                self.log('BUY EXECUTED, %.2f' % order.executed.price)         
            elif order.issell():             
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')     
        #Reset orders  
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))   
    
    def __init__(self):
        self.myind = ATR_TS_Long(self.data)
        self.crossover = btind.CrossOver(self.data.close, self.myind)
        
        self.order = None

    
    def next(self):
        
        #self.log('Close, %.2f' % self.data.close[0])
        
        if self.order:
            return
        
        if not self.position:
      
            if self.crossover[-1] > 0:
                p1 = self.data.close[0]
                p2 = self.data.close[0] + self.p.targ
                p3 = self.data.close[0] - self.p.stop
                self.order = self.buy_bracket(price=p1,limitprice=p2, stopprice=p3,)
                self.log('BUY CREATE, exectype Bracket, price %.2f, limitprice %.2f, stopprice %.2f' % 
                            (p1, p2, p3))

            elif self.crossover[-1] < 0:
                p1 = self.data.close[0]
                p2 = self.data.close[0] - self.p.stop
                p3 = self.data.close[0] + self.p.targ
                self.order = self.sell_bracket(price=p1,limitprice=p2,stopprice=p3,)
                self.log('SELL CREATE, exectype Bracket, price %.2f, limitprice %.2f, stopprice %.2f' %
                        (p1, p2, p3))

                    
        else: 
            if self.position:
                return
                
    def stop(self):
        if self.broker.getvalue()>10000000.0:
            self.log('(Target %3d), (Stop %3d), Ending Value %.2f' %
                (self.p.targ, self.p.stop,self.broker.getvalue()), doprint=True)
        
                    
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    data = bt.feeds.YahooFinanceCSVData(
    dataname='ES=F.csv',
    fromdate=datetime.datetime(2020, 1, 4),
    todate=datetime.datetime(2021, 1, 4),
    reverse=False)
    
    cerebro.adddata(data)
    
    cerebro.optstrategy(ATRTS1D, targ=range(100,1001,50), stop=range(50,501,50),)

    cerebro.addsizer(bt.sizers.SizerFix, stake=2500)
                     
    cerebro.broker.setcash(10000000.0)
    
    thestrat=cerebro.run(maxcpus=1)

2021-01-03, (Target 100), (Stop 300), Ending Value 10016875.00
2021-01-03, (Target 100), (Stop 350), Ending Value 10016875.00
2021-01-03, (Target 100), (Stop 400), Ending Value 10016875.00
2021-01-03, (Target 100), (Stop 450), Ending Value 10016875.00
2021-01-03, (Target 100), (Stop 500), Ending Value 10016875.00
2021-01-03, (Target 150), (Stop  50), Ending Value 10019375.00
2021-01-03, (Target 150), (Stop 300), Ending Value 10016875.00
2021-01-03, (Target 150), (Stop 350), Ending Value 10016875.00
2021-01-03, (Target 150), (Stop 400), Ending Value 10016875.00
2021-01-03, (Target 150), (Stop 450), Ending Value 10016875.00
2021-01-03, (Target 150), (Stop 500), Ending Value 10016875.00
2021-01-03, (Target 200), (Stop  50), Ending Value 10019375.00
2021-01-03, (Target 200), (Stop 100), Ending Value 10016875.00
2021-01-03, (Target 200), (Stop 150), Ending Value 10016875.00
2021-01-03, (Target 200), (Stop 200), Ending Value 10016875.00
2021-01-03, (Target 200), (Stop 250), Ending Value 1001